# Links:

Blowfish:
https://github.com/ananya2407/Blowfish-Algorithm

McEliece:
https://github.com/elibtronic/pythonMCS

EC DSA signature:
https://dev.to/exemak/bitcoin-signatures-from-scratch-44-ecdsa-implementation-in-python-using-zero-dependencies-410c


# installs

In [1]:
!pip install docopt
!pip install numpy
!pip install sympy
!pip install python-flint
!pip install pycryptodome

  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=5e39ec49aed4e149e70cc0c4ef18added0a5735727ea65478f6f2fd8adbbd841
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.7 MB/s eta 0:00:00


# Imports

In [2]:
import numpy as np
import numpy.polynomial.polynomial
from sympy.abc import x, alpha
from sympy import GF, Poly, Pow, Add, Symbol
from sympy.polys.galoistools import gf_irreducible, gf_irreducible_p
from sympy import lcm, ZZ
import math
import logging
import flint
from sympy import Matrix
import timeit
import hashlib, binascii, random
from Crypto.Random import get_random_bytes
import hashlib

# Blowfish
(converted into a class)

## util functions

In [3]:
def string_to_utf_integer(string, padding_length=4):
    # Join the ord() values of each character into a single string, padding each value
    utf_values = ''.join([str(ord(char)).zfill(padding_length) for char in string])

    # Convert the concatenated string of numbers into an integer
    return int(utf_values)


def utf_integer_to_string(utf_integer, padding_length=4):
    # Convert the integer to a string
    utf_values_str = str(utf_integer)

    # Pad the string to ensure it's a multiple of the padding length
    # If necessary, add leading zeros to match the required padding length
    padded_utf_values_str = utf_values_str.zfill(len(utf_values_str) + (padding_length - len(utf_values_str) % padding_length) % padding_length)

    # Split the string into chunks of padding_length
    char_codes = [int(padded_utf_values_str[i:i+padding_length]) for i in range(0, len(padded_utf_values_str), padding_length)]

    # Convert each chunk back into a character and join them into the final string
    return ''.join(chr(code) for code in char_codes)


def split_into_64bit_blocks(n):
    # List to store the blocks
    blocks = []

    # While there are bits left in the integer
    while n > 0:
        # Extract the lowest 64 bits of the integer (n & 0xFFFFFFFFFFFFFFFF masks out everything except the lowest 64 bits)
        block = n & 0xFFFFFFFFFFFFFFFF
        blocks.append(block)

        # Shift the integer right by 64 bits to process the next block
        n >>= 64

    return blocks

def combine_from_64bit_blocks(blocks):
    # Initialize the result as 0
    result = 0

    # Loop through each block in reverse order (from most significant to least)
    for block in reversed(blocks):
        # Shift the result left by 64 bits (to make space for the new block)
        result = (result << 64) | block

    return result

def generate_max_blowfish_key():
    key = get_random_bytes(56)  # 56 bytes = 448 bits

    # Split key into 32-bit (4-byte) blocks
    blocks_32bit = [int.from_bytes(key[i:i+4], 'big') for i in range(0, len(key), 4)]

    return key, blocks_32bit


## algorithem

In [4]:
class Blowfish:

  def __init__(self, key_blocks=[ 0x4B7A70E9, 0xB5B32944, 0xDB75092E, 0xC4192623,
            0xAD6EA6B0, 0x49A7DF7D, 0x9CEE60B8, 0x8FEDB266,
            0xECAA8C71, 0x699A17FF, 0x5664526C, 0xC2B19EE1,
            0x193602A5, 0x75094C29]):
    self.key = key_blocks
    self.p = [
      0x243F6A88, 0x85A308D3, 0x13198A2E, 0x03707344,
      0xA4093822, 0x299F31D0, 0x082EFA98, 0xEC4E6C89,
      0x452821E6, 0x38D01377, 0xBE5466CF, 0x34E90C6C,
      0xC0AC29B7, 0xC97C50DD, 0x3F84D5B5, 0xB5470917,
      0x9216D5D9, 0x8979FB1B
      ]

    self.s = [
          [
              0xD1310BA6, 0x98DFB5AC, 0x2FFD72DB, 0xD01ADFB7,
              0xB8E1AFED, 0x6A267E96, 0xBA7C9045, 0xF12C7F99,
              0x24A19947, 0xB3916CF7, 0x0801F2E2, 0x858EFC16,
              0x636920D8, 0x71574E69, 0xA458FEA3, 0xF4933D7E,
              0x0D95748F, 0x728EB658, 0x718BCD58, 0x82154AEE,
              0x7B54A41D, 0xC25A59B5, 0x9C30D539, 0x2AF26013,
              0xC5D1B023, 0x286085F0, 0xCA417918, 0xB8DB38EF,
              0x8E79DCB0, 0x603A180E, 0x6C9E0E8B, 0xB01E8A3E,
              0xD71577C1, 0xBD314B27, 0x78AF2FDA, 0x55605C60,
              0xE65525F3, 0xAA55AB94, 0x57489862, 0x63E81440,
              0x55CA396A, 0x2AAB10B6, 0xB4CC5C34, 0x1141E8CE,
              0xA15486AF, 0x7C72E993, 0xB3EE1411, 0x636FBC2A,
              0x2BA9C55D, 0x741831F6, 0xCE5C3E16, 0x9B87931E,
              0xAFD6BA33, 0x6C24CF5C, 0x7A325381, 0x28958677,
              0x3B8F4898, 0x6B4BB9AF, 0xC4BFE81B, 0x66282193,
              0x61D809CC, 0xFB21A991, 0x487CAC60, 0x5DEC8032,
              0xEF845D5D, 0xE98575B1, 0xDC262302, 0xEB651B88,
              0x23893E81, 0xD396ACC5, 0x0F6D6FF3, 0x83F44239,
              0x2E0B4482, 0xA4842004, 0x69C8F04A, 0x9E1F9B5E,
              0x21C66842, 0xF6E96C9A, 0x670C9C61, 0xABD388F0,
              0x6A51A0D2, 0xD8542F68, 0x960FA728, 0xAB5133A3,
              0x6EEF0B6C, 0x137A3BE4, 0xBA3BF050, 0x7EFB2A98,
              0xA1F1651D, 0x39AF0176, 0x66CA593E, 0x82430E88,
              0x8CEE8619, 0x456F9FB4, 0x7D84A5C3, 0x3B8B5EBE,
              0xE06F75D8, 0x85C12073, 0x401A449F, 0x56C16AA6,
              0x4ED3AA62, 0x363F7706, 0x1BFEDF72, 0x429B023D,
              0x37D0D724, 0xD00A1248, 0xDB0FEAD3, 0x49F1C09B,
              0x075372C9, 0x80991B7B, 0x25D479D8, 0xF6E8DEF7,
              0xE3FE501A, 0xB6794C3B, 0x976CE0BD, 0x04C006BA,
              0xC1A94FB6, 0x409F60C4, 0x5E5C9EC2, 0x196A2463,
              0x68FB6FAF, 0x3E6C53B5, 0x1339B2EB, 0x3B52EC6F,
              0x6DFC511F, 0x9B30952C, 0xCC814544, 0xAF5EBD09,
              0xBEE3D004, 0xDE334AFD, 0x660F2807, 0x192E4BB3,
              0xC0CBA857, 0x45C8740F, 0xD20B5F39, 0xB9D3FBDB,
              0x5579C0BD, 0x1A60320A, 0xD6A100C6, 0x402C7279,
              0x679F25FE, 0xFB1FA3CC, 0x8EA5E9F8, 0xDB3222F8,
              0x3C7516DF, 0xFD616B15, 0x2F501EC8, 0xAD0552AB,
              0x323DB5FA, 0xFD238760, 0x53317B48, 0x3E00DF82,
              0x9E5C57BB, 0xCA6F8CA0, 0x1A87562E, 0xDF1769DB,
              0xD542A8F6, 0x287EFFC3, 0xAC6732C6, 0x8C4F5573,
              0x695B27B0, 0xBBCA58C8, 0xE1FFA35D, 0xB8F011A0,
              0x10FA3D98, 0xFD2183B8, 0x4AFCB56C, 0x2DD1D35B,
              0x9A53E479, 0xB6F84565, 0xD28E49BC, 0x4BFB9790,
              0xE1DDF2DA, 0xA4CB7E33, 0x62FB1341, 0xCEE4C6E8,
              0xEF20CADA, 0x36774C01, 0xD07E9EFE, 0x2BF11FB4,
              0x95DBDA4D, 0xAE909198, 0xEAAD8E71, 0x6B93D5A0,
              0xD08ED1D0, 0xAFC725E0, 0x8E3C5B2F, 0x8E7594B7,
              0x8FF6E2FB, 0xF2122B64, 0x8888B812, 0x900DF01C,
              0x4FAD5EA0, 0x688FC31C, 0xD1CFF191, 0xB3A8C1AD,
              0x2F2F2218, 0xBE0E1777, 0xEA752DFE, 0x8B021FA1,
              0xE5A0CC0F, 0xB56F74E8, 0x18ACF3D6, 0xCE89E299,
              0xB4A84FE0, 0xFD13E0B7, 0x7CC43B81, 0xD2ADA8D9,
              0x165FA266, 0x80957705, 0x93CC7314, 0x211A1477,
              0xE6AD2065, 0x77B5FA86, 0xC75442F5, 0xFB9D35CF,
              0xEBCDAF0C, 0x7B3E89A0, 0xD6411BD3, 0xAE1E7E49,
              0x00250E2D, 0x2071B35E, 0x226800BB, 0x57B8E0AF,
              0x2464369B, 0xF009B91E, 0x5563911D, 0x59DFA6AA,
              0x78C14389, 0xD95A537F, 0x207D5BA2, 0x02E5B9C5,
              0x83260376, 0x6295CFA9, 0x11C81968, 0x4E734A41,
              0xB3472DCA, 0x7B14A94A, 0x1B510052, 0x9A532915,
              0xD60F573F, 0xBC9BC6E4, 0x2B60A476, 0x81E67400,
              0x08BA6FB5, 0x571BE91F, 0xF296EC6B, 0x2A0DD915,
              0xB6636521, 0xE7B9F9B6, 0xFF34052E, 0xC5855664,
              0x53B02D5D, 0xA99F8FA1, 0x08BA4799, 0x6E85076A
          ],
          [
              0x4B7A70E9, 0xB5B32944, 0xDB75092E, 0xC4192623,
              0xAD6EA6B0, 0x49A7DF7D, 0x9CEE60B8, 0x8FEDB266,
              0xECAA8C71, 0x699A17FF, 0x5664526C, 0xC2B19EE1,
              0x193602A5, 0x75094C29, 0xA0591340, 0xE4183A3E,
              0x3F54989A, 0x5B429D65, 0x6B8FE4D6, 0x99F73FD6,
              0xA1D29C07, 0xEFE830F5, 0x4D2D38E6, 0xF0255DC1,
              0x4CDD2086, 0x8470EB26, 0x6382E9C6, 0x021ECC5E,
              0x09686B3F, 0x3EBAEFC9, 0x3C971814, 0x6B6A70A1,
              0x687F3584, 0x52A0E286, 0xB79C5305, 0xAA500737,
              0x3E07841C, 0x7FDEAE5C, 0x8E7D44EC, 0x5716F2B8,
              0xB03ADA37, 0xF0500C0D, 0xF01C1F04, 0x0200B3FF,
              0xAE0CF51A, 0x3CB574B2, 0x25837A58, 0xDC0921BD,
              0xD19113F9, 0x7CA92FF6, 0x94324773, 0x22F54701,
              0x3AE5E581, 0x37C2DADC, 0xC8B57634, 0x9AF3DDA7,
              0xA9446146, 0x0FD0030E, 0xECC8C73E, 0xA4751E41,
              0xE238CD99, 0x3BEA0E2F, 0x3280BBA1, 0x183EB331,
              0x4E548B38, 0x4F6DB908, 0x6F420D03, 0xF60A04BF,
              0x2CB81290, 0x24977C79, 0x5679B072, 0xBCAF89AF,
              0xDE9A771F, 0xD9930810, 0xB38BAE12, 0xDCCF3F2E,
              0x5512721F, 0x2E6B7124, 0x501ADDE6, 0x9F84CD87,
              0x7A584718, 0x7408DA17, 0xBC9F9ABC, 0xE94B7D8C,
              0xEC7AEC3A, 0xDB851DFA, 0x63094366, 0xC464C3D2,
              0xEF1C1847, 0x3215D908, 0xDD433B37, 0x24C2BA16,
              0x12A14D43, 0x2A65C451, 0x50940002, 0x133AE4DD,
              0x71DFF89E, 0x10314E55, 0x81AC77D6, 0x5F11199B,
              0x043556F1, 0xD7A3C76B, 0x3C11183B, 0x5924A509,
              0xF28FE6ED, 0x97F1FBFA, 0x9EBABF2C, 0x1E153C6E,
              0x86E34570, 0xEAE96FB1, 0x860E5E0A, 0x5A3E2AB3,
              0x771FE71C, 0x4E3D06FA, 0x2965DCB9, 0x99E71D0F,
              0x803E89D6, 0x5266C825, 0x2E4CC978, 0x9C10B36A,
              0xC6150EBA, 0x94E2EA78, 0xA5FC3C53, 0x1E0A2DF4,
              0xF2F74EA7, 0x361D2B3D, 0x1939260F, 0x19C27960,
              0x5223A708, 0xF71312B6, 0xEBADFE6E, 0xEAC31F66,
              0xE3BC4595, 0xA67BC883, 0xB17F37D1, 0x018CFF28,
              0xC332DDEF, 0xBE6C5AA5, 0x65582185, 0x68AB9802,
              0xEECEA50F, 0xDB2F953B, 0x2AEF7DAD, 0x5B6E2F84,
              0x1521B628, 0x29076170, 0xECDD4775, 0x619F1510,
              0x13CCA830, 0xEB61BD96, 0x0334FE1E, 0xAA0363CF,
              0xB5735C90, 0x4C70A239, 0xD59E9E0B, 0xCBAADE14,
              0xEECC86BC, 0x60622CA7, 0x9CAB5CAB, 0xB2F3846E,
              0x648B1EAF, 0x19BDF0CA, 0xA02369B9, 0x655ABB50,
              0x40685A32, 0x3C2AB4B3, 0x319EE9D5, 0xC021B8F7,
              0x9B540B19, 0x875FA099, 0x95F7997E, 0x623D7DA8,
              0xF837889A, 0x97E32D77, 0x11ED935F, 0x16681281,
              0x0E358829, 0xC7E61FD6, 0x96DEDFA1, 0x7858BA99,
              0x57F584A5, 0x1B227263, 0x9B83C3FF, 0x1AC24696,
              0xCDB30AEB, 0x532E3054, 0x8FD948E4, 0x6DBC3128,
              0x58EBF2EF, 0x34C6FFEA, 0xFE28ED61, 0xEE7C3C73,
              0x5D4A14D9, 0xE864B7E3, 0x42105D14, 0x203E13E0,
              0x45EEE2B6, 0xA3AAABEA, 0xDB6C4F15, 0xFACB4FD0,
              0xC742F442, 0xEF6ABBB5, 0x654F3B1D, 0x41CD2105,
              0xD81E799E, 0x86854DC7, 0xE44B476A, 0x3D816250,
              0xCF62A1F2, 0x5B8D2646, 0xFC8883A0, 0xC1C7B6A3,
              0x7F1524C3, 0x69CB7492, 0x47848A0B, 0x5692B285,
              0x095BBF00, 0xAD19489D, 0x1462B174, 0x23820E00,
              0x58428D2A, 0x0C55F5EA, 0x1DADF43E, 0x233F7061,
              0x3372F092, 0x8D937E41, 0xD65FECF1, 0x6C223BDB,
              0x7CDE3759, 0xCBEE7460, 0x4085F2A7, 0xCE77326E,
              0xA6078084, 0x19F8509E, 0xE8EFD855, 0x61D99735,
              0xA969A7AA, 0xC50C06C2, 0x5A04ABFC, 0x800BCADC,
              0x9E447A2E, 0xC3453484, 0xFDD56705, 0x0E1E9EC9,
              0xDB73DBD3, 0x105588CD, 0x675FDA79, 0xE3674340,
              0xC5C43465, 0x713E38D8, 0x3D28F89E, 0xF16DFF20,
              0x153E21E7, 0x8FB03D4A, 0xE6E39F2B, 0xDB83ADF7
          ],
          [
              0xE93D5A68, 0x948140F7, 0xF64C261C, 0x94692934,
              0x411520F7, 0x7602D4F7, 0xBCF46B2E, 0xD4A20068,
              0xD4082471, 0x3320F46A, 0x43B7D4B7, 0x500061AF,
              0x1E39F62E, 0x97244546, 0x14214F74, 0xBF8B8840,
              0x4D95FC1D, 0x96B591AF, 0x70F4DDD3, 0x66A02F45,
              0xBFBC09EC, 0x03BD9785, 0x7FAC6DD0, 0x31CB8504,
              0x96EB27B3, 0x55FD3941, 0xDA2547E6, 0xABCA0A9A,
              0x28507825, 0x530429F4, 0x0A2C86DA, 0xE9B66DFB,
              0x68DC1462, 0xD7486900, 0x680EC0A4, 0x27A18DEE,
              0x4F3FFEA2, 0xE887AD8C, 0xB58CE006, 0x7AF4D6B6,
              0xAACE1E7C, 0xD3375FEC, 0xCE78A399, 0x406B2A42,
              0x20FE9E35, 0xD9F385B9, 0xEE39D7AB, 0x3B124E8B,
              0x1DC9FAF7, 0x4B6D1856, 0x26A36631, 0xEAE397B2,
              0x3A6EFA74, 0xDD5B4332, 0x6841E7F7, 0xCA7820FB,
              0xFB0AF54E, 0xD8FEB397, 0x454056AC, 0xBA489527,
              0x55533A3A, 0x20838D87, 0xFE6BA9B7, 0xD096954B,
              0x55A867BC, 0xA1159A58, 0xCCA92963, 0x99E1DB33,
              0xA62A4A56, 0x3F3125F9, 0x5EF47E1C, 0x9029317C,
              0xFDF8E802, 0x04272F70, 0x80BB155C, 0x05282CE3,
              0x95C11548, 0xE4C66D22, 0x48C1133F, 0xC70F86DC,
              0x07F9C9EE, 0x41041F0F, 0x404779A4, 0x5D886E17,
              0x325F51EB, 0xD59BC0D1, 0xF2BCC18F, 0x41113564,
              0x257B7834, 0x602A9C60, 0xDFF8E8A3, 0x1F636C1B,
              0x0E12B4C2, 0x02E1329E, 0xAF664FD1, 0xCAD18115,
              0x6B2395E0, 0x333E92E1, 0x3B240B62, 0xEEBEB922,
              0x85B2A20E, 0xE6BA0D99, 0xDE720C8C, 0x2DA2F728,
              0xD0127845, 0x95B794FD, 0x647D0862, 0xE7CCF5F0,
              0x5449A36F, 0x877D48FA, 0xC39DFD27, 0xF33E8D1E,
              0x0A476341, 0x992EFF74, 0x3A6F6EAB, 0xF4F8FD37,
              0xA812DC60, 0xA1EBDDF8, 0x991BE14C, 0xDB6E6B0D,
              0xC67B5510, 0x6D672C37, 0x2765D43B, 0xDCD0E804,
              0xF1290DC7, 0xCC00FFA3, 0xB5390F92, 0x690FED0B,
              0x667B9FFB, 0xCEDB7D9C, 0xA091CF0B, 0xD9155EA3,
              0xBB132F88, 0x515BAD24, 0x7B9479BF, 0x763BD6EB,
              0x37392EB3, 0xCC115979, 0x8026E297, 0xF42E312D,
              0x6842ADA7, 0xC66A2B3B, 0x12754CCC, 0x782EF11C,
              0x6A124237, 0xB79251E7, 0x06A1BBE6, 0x4BFB6350,
              0x1A6B1018, 0x11CAEDFA, 0x3D25BDD8, 0xE2E1C3C9,
              0x44421659, 0x0A121386, 0xD90CEC6E, 0xD5ABEA2A,
              0x64AF674E, 0xDA86A85F, 0xBEBFE988, 0x64E4C3FE,
              0x9DBC8057, 0xF0F7C086, 0x60787BF8, 0x6003604D,
              0xD1FD8346, 0xF6381FB0, 0x7745AE04, 0xD736FCCC,
              0x83426B33, 0xF01EAB71, 0xB0804187, 0x3C005E5F,
              0x77A057BE, 0xBDE8AE24, 0x55464299, 0xBF582E61,
              0x4E58F48F, 0xF2DDFDA2, 0xF474EF38, 0x8789BDC2,
              0x5366F9C3, 0xC8B38E74, 0xB475F255, 0x46FCD9B9,
              0x7AEB2661, 0x8B1DDF84, 0x846A0E79, 0x915F95E2,
              0x466E598E, 0x20B45770, 0x8CD55591, 0xC902DE4C,
              0xB90BACE1, 0xBB8205D0, 0x11A86248, 0x7574A99E,
              0xB77F19B6, 0xE0A9DC09, 0x662D09A1, 0xC4324633,
              0xE85A1F02, 0x09F0BE8C, 0x4A99A025, 0x1D6EFE10,
              0x1AB93D1D, 0x0BA5A4DF, 0xA186F20F, 0x2868F169,
              0xDCB7DA83, 0x573906FE, 0xA1E2CE9B, 0x4FCD7F52,
              0x50115E01, 0xA70683FA, 0xA002B5C4, 0x0DE6D027,
              0x9AF88C27, 0x773F8641, 0xC3604C06, 0x61A806B5,
              0xF0177A28, 0xC0F586E0, 0x006058AA, 0x30DC7D62,
              0x11E69ED7, 0x2338EA63, 0x53C2DD94, 0xC2C21634,
              0xBBCBEE56, 0x90BCB6DE, 0xEBFC7DA1, 0xCE591D76,
              0x6F05E409, 0x4B7C0188, 0x39720A3D, 0x7C927C24,
              0x86E3725F, 0x724D9DB9, 0x1AC15BB4, 0xD39EB8FC,
              0xED545578, 0x08FCA5B5, 0xD83D7CD3, 0x4DAD0FC4,
              0x1E50EF5E, 0xB161E6F8, 0xA28514D9, 0x6C51133C,
              0x6FD5C7E7, 0x56E14EC4, 0x362ABFCE, 0xDDC6C837,
              0xD79A3234, 0x92638212, 0x670EFA8E, 0x406000E0
          ],
          [
              0x3A39CE37, 0xD3FAF5CF, 0xABC27737, 0x5AC52D1B,
              0x5CB0679E, 0x4FA33742, 0xD3822740, 0x99BC9BBE,
              0xD5118E9D, 0xBF0F7315, 0xD62D1C7E, 0xC700C47B,
              0xB78C1B6B, 0x21A19045, 0xB26EB1BE, 0x6A366EB4,
              0x5748AB2F, 0xBC946E79, 0xC6A376D2, 0x6549C2C8,
              0x530FF8EE, 0x468DDE7D, 0xD5730A1D, 0x4CD04DC6,
              0x2939BBDB, 0xA9BA4650, 0xAC9526E8, 0xBE5EE304,
              0xA1FAD5F0, 0x6A2D519A, 0x63EF8CE2, 0x9A86EE22,
              0xC089C2B8, 0x43242EF6, 0xA51E03AA, 0x9CF2D0A4,
              0x83C061BA, 0x9BE96A4D, 0x8FE51550, 0xBA645BD6,
              0x2826A2F9, 0xA73A3AE1, 0x4BA99586, 0xEF5562E9,
              0xC72FEFD3, 0xF752F7DA, 0x3F046F69, 0x77FA0A59,
              0x80E4A915, 0x87B08601, 0x9B09E6AD, 0x3B3EE593,
              0xE990FD5A, 0x9E34D797, 0x2CF0B7D9, 0x022B8B51,
              0x96D5AC3A, 0x017DA67D, 0xD1CF3ED6, 0x7C7D2D28,
              0x1F9F25CF, 0xADF2B89B, 0x5AD6B472, 0x5A88F54C,
              0xE029AC71, 0xE019A5E6, 0x47B0ACFD, 0xED93FA9B,
              0xE8D3C48D, 0x283B57CC, 0xF8D56629, 0x79132E28,
              0x785F0191, 0xED756055, 0xF7960E44, 0xE3D35E8C,
              0x15056DD4, 0x88F46DBA, 0x03A16125, 0x0564F0BD,
              0xC3EB9E15, 0x3C9057A2, 0x97271AEC, 0xA93A072A,
              0x1B3F6D9B, 0x1E6321F5, 0xF59C66FB, 0x26DCF319,
              0x7533D928, 0xB155FDF5, 0x03563482, 0x8ABA3CBB,
              0x28517711, 0xC20AD9F8, 0xABCC5167, 0xCCAD925F,
              0x4DE81751, 0x3830DC8E, 0x379D5862, 0x9320F991,
              0xEA7A90C2, 0xFB3E7BCE, 0x5121CE64, 0x774FBE32,
              0xA8B6E37E, 0xC3293D46, 0x48DE5369, 0x6413E680,
              0xA2AE0810, 0xDD6DB224, 0x69852DFD, 0x09072166,
              0xB39A460A, 0x6445C0DD, 0x586CDECF, 0x1C20C8AE,
              0x5BBEF7DD, 0x1B588D40, 0xCCD2017F, 0x6BB4E3BB,
              0xDDA26A7E, 0x3A59FF45, 0x3E350A44, 0xBCB4CDD5,
              0x72EACEA8, 0xFA6484BB, 0x8D6612AE, 0xBF3C6F47,
              0xD29BE463, 0x542F5D9E, 0xAEC2771B, 0xF64E6370,
              0x740E0D8D, 0xE75B1357, 0xF8721671, 0xAF537D5D,
              0x4040CB08, 0x4EB4E2CC, 0x34D2466A, 0x0115AF84,
              0xE1B00428, 0x95983A1D, 0x06B89FB4, 0xCE6EA048,
              0x6F3F3B82, 0x3520AB82, 0x011A1D4B, 0x277227F8,
              0x611560B1, 0xE7933FDC, 0xBB3A792B, 0x344525BD,
              0xA08839E1, 0x51CE794B, 0x2F32C9B7, 0xA01FBAC9,
              0xE01CC87E, 0xBCC7D1F6, 0xCF0111C3, 0xA1E8AAC7,
              0x1A908749, 0xD44FBD9A, 0xD0DADECB, 0xD50ADA38,
              0x0339C32A, 0xC6913667, 0x8DF9317C, 0xE0B12B4F,
              0xF79E59B7, 0x43F5BB3A, 0xF2D519FF, 0x27D9459C,
              0xBF97222C, 0x15E6FC2A, 0x0F91FC71, 0x9B941525,
              0xFAE59361, 0xCEB69CEB, 0xC2A86459, 0x12BAA8D1,
              0xB6C1075E, 0xE3056A0C, 0x10D25065, 0xCB03A442,
              0xE0EC6E0E, 0x1698DB3B, 0x4C98A0BE, 0x3278E964,
              0x9F1F9532, 0xE0D392DF, 0xD3A0342B, 0x8971F21E,
              0x1B0A7441, 0x4BA3348C, 0xC5BE7120, 0xC37632D8,
              0xDF359F8D, 0x9B992F2E, 0xE60B6F47, 0x0FE3F11D,
              0xE54CDA54, 0x1EDAD891, 0xCE6279CF, 0xCD3E7E6F,
              0x1618B166, 0xFD2C1D05, 0x848FD2C5, 0xF6FB2299,
              0xF523F357, 0xA6327623, 0x93A83531, 0x56CCCD02,
              0xACF08162, 0x5A75EBB5, 0x6E163697, 0x88D273CC,
              0xDE966292, 0x81B949D0, 0x4C50901B, 0x71C65614,
              0xE6C6C7BD, 0x327A140A, 0x45E1D006, 0xC3F27B9A,
              0xC9AA53FD, 0x62A80F00, 0xBB25BFE2, 0x35BDD2F6,
              0x71126905, 0xB2040222, 0xB6CBCF7C, 0xCD769C2B,
              0x53113EC0, 0x1640E3D3, 0x38ABBD60, 0x2547ADF0,
              0xBA38209C, 0xF746CE76, 0x77AFA1C5, 0x20756060,
              0x85CBFE4E, 0x8AE88DD8, 0x7AAAF9B0, 0x4CF9AA7E,
              0x1948C25C, 0x02FB8A8C, 0x01C36AE4, 0xD6EBE1F9,
              0x90D4F869, 0xA65CDEA0, 0x3F09252D, 0xC208E69F,
              0xB74E6132, 0xCE77E25B, 0x578FDFE3, 0x3AC372E6
          ]
    ]

    # uses the key for the p boxes
    for i in range(0,18):
        self.p[i] = self.p[i]^self.key[i%14]

  def calculate(self, L):
      temp = self.s[0][L >> 24]
      temp = (temp + self.s[1][L >> 16 & 0xff]) % (0x1<<32)
      temp = temp ^ self.s[2][L >> 8 & 0xff]
      temp = (temp + self.s[3][L & 0xff]) % (0x1<<32)
      return temp

  def encrypt(self, data):
        L = data>>32
        R = data & 0xffffffff
        for i in range(0,16):
                L = L^self.p[i]
                L1 = self.calculate(L)
                R = R^self.calculate(L1)
                L,R = R,L
        L,R = R,L
        L = L^self.p[17]
        R = R^self.p[16]
        encrypted = (L<<32) ^ R
        return encrypted

  def decrypt(self, data):
        L = data >> 32
        R = data & 0xffffffff
        for i in range(17, 1, -1):
            L = self.p[i]^L
            L1 = self.calculate(L)
            R = R^self.calculate(L1)
            L,R = R,L

        L,R = R,L
        L = L^self.p[0]
        R = R^self.p[1]
        data_decrypted1 = (L<<32) ^ R
        return data_decrypted1

  # added by us for easier use

  def encrypt_str(self, msg):
    msg_bits = string_to_utf_integer(msg)
    msg_blocks = split_into_64bit_blocks(msg_bits)
    encrypted_blocks = []
    for b in msg_blocks:
      data_encrypted = self.encrypt(b)
      encrypted_blocks.append(data_encrypted)
    encrypted_bits = combine_from_64bit_blocks(encrypted_blocks)
    return utf_integer_to_string(encrypted_bits)

  def decrypt_str(self, msg):
    msg_bits = string_to_utf_integer(msg)
    msg_blocks = split_into_64bit_blocks(msg_bits)
    decrypted_blocks = []
    for b in msg_blocks:
      data_decrypted = self.decrypt(b)
      decrypted_blocks.append(data_decrypted)
    decrypted_bits = combine_from_64bit_blocks(decrypted_blocks)
    return utf_integer_to_string(decrypted_bits)


## examples for Blowfish algorithem

example with dynamic key

In [5]:
key,key_blocks = generate_max_blowfish_key()
print(f"Key={key}")
print(f"Key_blocks={key_blocks}")
msg = "Hello World!"
bf = Blowfish(key_blocks=key_blocks)
en = bf.encrypt_str(msg)
print(f"message: {msg}")
print(f"Encrypted message: {en}")
print(f"Decrypted message= {bf.decrypt_str(en)}")

Key=b'va%\xba\xdb\xd7 ~\xe3v\xfa<\xa9p\xbeq4Q\xde?M\xb77\xf7\x06\x13\xee\x0b\xcdk7~\xcf\x19V\x16\xa4\x10q,\x8b\xa35\xc4.\xdd\xc2X[h;\xda\x9e\xeb{\x8e'
Key_blocks=[1986078138, 3688308862, 3816225340, 2842738289, 877780543, 1303853047, 101969419, 3446355838, 3474544150, 2752540972, 2342729156, 786285144, 1533557722, 2666232718]
message: Hello World!
Encrypted message: ᚉ␥₳ۯ̮ጤᅑތಳ౪ଟਂẫ᥵
Decrypted message= Hello World!


example with prints

In [6]:
# bf = Blowfish()
# # for i in range(0,18):
# #     bf.p[i] = bf.p[i]^bf.key[i%14]
# # x = 0
# # data = 0
# # for i in range(0,9):
# #     temp = bf.encrypt(data)
# #     bf.p[x] = temp >> 32
# #     x+=1
# #     bf.p[x] = temp & 0xffffffff
# #     x+=1
# #     data = temp

# encrypt_data = input("Enter data to encrypt with Blowfish: ")
# encrypt_data_utf = string_to_utf_integer(encrypt_data)
# blocks = split_into_64bit_blocks(encrypt_data_utf)
# print(f"UTF-Data: {encrypt_data_utf}")
# print(f"Blocks: {blocks}")
# res_e = []
# for b in blocks:
#   print()
#   if b.bit_length() <=64:
#     print(f"Valid Input!!! {b}")
#   else:
#       print("Invalid Input!!")

#   data_encrypted = bf.encrypt(b)
#   print("Encrypted data is: ",data_encrypted)
#   print("Hex value :",hex(data_encrypted))
#   res_e.append(data_encrypted)

# print(f"Encrypted blocks: {res_e}")
# en_data = combine_from_64bit_blocks(res_e)
# print(f"UTF-data: {en_data}")
# print(f"String: {utf_integer_to_string(en_data)}\n")

# res_c = []
# for e in res_e:
#   print()
#   data_decrypted = bf.decrypt(e)
#   print("Data after decryption is : ",data_decrypted)
#   print("Data str after decryption is : ",utf_integer_to_string(data_decrypted))
#   res_c.append(data_decrypted)

# print(f"\nDecrypted blocks: {res_c}")
# de_data = combine_from_64bit_blocks(res_c)
# print(f"UTF-data: {de_data}")
# print(f"String: {utf_integer_to_string(de_data)}")

# McEliece
https://github.com/elibtronic/pythonMCS

## algorithm

In [7]:
# Core Library Utilities
import numpy as np
import pickle
import os.path
import random
import csv


###### Helper Functions #######
def genSMatrix(k):
    """Generates an invertable matrix k * k"""
    sMaybe = np.matrix(np.random.randint(0, 2, k * k).reshape(k, k).astype(int))
    while True:
        try:
            sMaybe.getI()
            return sMaybe
        except:
            sMaybe = np.matrix(np.random.randint(0, 2, k * k).reshape(k, k).astype(int))


def genPMatrix(n, keep=False):
    """Generates a permutation matrix n x n using the given sequence"""
    p = np.identity(n, dtype=int)
    if keep:
        return np.matrix(p).reshape(n, n)
    else:
        return np.matrix(np.random.permutation(p))


def modTwo(C):
    """Q & D way to Mod 2 all results"""
    D = C.copy()
    D.fill(2)
    return np.remainder(C, D)


def bitFlip(C, n):
    """Flips the bit you tell it, -1 for random bit flip, 0 for no flip"""
    if n == 0:
        return C
    if n == -1:
        index = random.randint(1, C.size - 1)
    else:
        index = n

    if C[0, index - 1] == 1:
        C[0, index - 1] = 0
    else:
        C[0, index - 1] = 1
    return C


def all_zeros(d):
    """Used when looking up a Syndrome, all zeros mean no error"""
    zc = 0
    for x in d:
        if x == 0:
            zc += 1
    if zc == len(d):
        return True
    else:
        return False


def syndromeLookup(H, d):
    """Looks up where in the syndrome table, if not found must be codeword, in that case returns 0"""
    t = H.T.tolist()
    s = d.T.tolist()[0]

    # print( "s s\t",s

    if all_zeros(s):
        # print( "s i\t 0"
        return 0
    try:
        # print( "s i\t ",t.index(s) + 1
        return t.index(s) + 1
    except:
        # print( "s i\t 0"
        return 0


def checkOldGuesses(oG, newGuess):
    """Helper function to see if a guessed matrix has been used before"""

    for s in oG:
        if np.array_equal(newGuess.A1, s.A1):
            return False
        else:
            return True


def makeString(matrix):
    """Used in DNA file encoding, will make Numpy Matrix into text strings for easier manipulation"""
    message = ""
    for m in matrix.A1:
        if m == "[":
            pass
        elif m == "]":
            pass
        elif m == " ":
            pass
        else:
            message += str(m)
    return message


#### Hamming 8,4 Encryption Data Structures ######

#### Private Key H84 ####
class privateKeyH84:
    """Datastructure to represent our Private Key"""

    def __init__(self, S=None, P=None):
        """Initalizer that will set S & P matricies to random if not given values"""
        # Hamming 8,4 in standard
        self.G = np.matrix([
            [1, 0, 0, 0, 0, 1, 1, 1],
            [0, 1, 0, 0, 1, 0, 1, 1],
            [0, 0, 1, 0, 1, 1, 0, 1],
            [0, 0, 0, 1, 1, 1, 1, 0]
        ], dtype=int)
        self.H = np.matrix([
            [0, 1, 1, 1, 1, 0, 0, 0],
            [1, 0, 1, 1, 0, 1, 0, 0],
            [1, 1, 0, 1, 0, 0, 1, 0],
            [1, 1, 1, 0, 0, 0, 0, 1]
        ], dtype=int)

        # Can create these from known values, otherwise random
        if S == None:
            self.S = modTwo(genSMatrix(4))
        else:
            self.S = S

        if P == None:
            self.P = modTwo(genPMatrix(8))
        else:
            self.P = P

    def printCode(self):
        """Canonical print( to screen function"""
        print("S: \n" + str(self.S) + "\n")
        print("P: \n" + str(self.P) + "\n")
        print("GPrime: \n" + str(self.makeGPrime()) + "\n")

    def writeKeyToFile(self, keyFile):
        """Saves key to a pickle file"""
        try:
            pickle.dump(self, open(keyFile, "wb"))
        except:
            print("Could not save key file to: ", keyFile)
            exit(1)

    def readKeyFromFile(self, keyFile):
        """Reads key from a pickle file"""
        try:
            newPriv = pickle.load(open(keyFile, "rb"))
            self.S = newPriv.S
            self.P = newPriv.P
        except:
            print("Could not load key file from: ", keyFile)
            exit(1)

    def makeGPrime(self):
        """Creates the GPrime encrytion Matrix"""
        return modTwo(self.S * self.G * self.P)

    def decrypt(self, c):
        """When given cipher text will decode to message"""
        cHat = c * modTwo(self.P.I.astype(int))
        m = bitFlip(cHat, syndromeLookup(self.H, modTwo(self.H * cHat.T)))
        return modTwo(m[0, 0:4] * modTwo(self.S.I.astype(int)))

    def decryptFile(self, f):
        """Will decrypt whole file"""
        cf = open(f, "rb")
        cb1 = cf.read(1)
        cb2 = cf.read(1)

        mf = open(f + ".decoded", "wb")

        while cb1 and cb2:

            # First Byte of Cipher Text
            c_1 = '{0:08b}'.format(ord(cb1))[0:8]
            c1_l = []
            m1 = ""
            for s in c_1:
                c1_l.append(s)
            c_1_m = np.matrix(c1_l, dtype=int)

            d1 = self.decrypt(c_1_m)

            for d in range(0, d1.size):
                m1 += str(d1.item(d))

            # Second Byte of Cipher Text
            c_2 = '{0:08b}'.format(ord(cb2))[0:8]
            c2_l = []
            m2 = ""
            for s in c_2:
                c2_l.append(s)
            c_2_m = np.matrix(c2_l, dtype=int)

            d2 = self.decrypt(c_2_m)

            for d in range(0, d2.size):
                m2 += str(d2.item(d))

            # ( m1+m2
            mf.write(chr(int(m1 + m2, 2)))
            cb1 = cf.read(1)
            cb2 = cf.read(1)

        mf.close()
        cf.close()

    def dnaFileDecrypt(self, f, dlu):
        """Decrypts a file that has been turned into a DNA representation"""
        cf = open(f, "r")
        c1 = cf.readline().strip("\n")
        c2 = cf.readline().strip("\n")

        mf = open(f + ".decoded", "w")

        while c1 and c2:
            m1 = ""
            m2 = ""
            mat1 = np.matrix(" ".join(dlu.lookDNADecrypt(c1)), dtype=int)
            mat2 = np.matrix(" ".join(dlu.lookDNADecrypt(c2)), dtype=int)
            d1 = self.decrypt(mat1)
            d2 = self.decrypt(mat2)
            for d in range(0, d1.size):
                m1 += str(d1.item(d))

            for d in range(0, d2.size):
                m2 += str(d2.item(d))

            mf.write(chr(int(m1 + m2, 2)))
            c1 = cf.readline().strip("\n")
            c2 = cf.readline().strip("\n")

        mf.close()
        cf.close()


#### Public Key H84 ####
class publicKeyH84:
    """Public Key Data Structure"""

    def __init__(self, GPrime):
        self.GPrime = GPrime

    def printCode(self):
        """Canonical print( to screen"""
        print("GPrime: \n" + str(self.GPrime) + "\n")

    def writeKeyToFile(self, keyFile):
        """Saves key to a pickle file"""
        try:
            pickle.dump(self, open(keyFile, "wb"))
        except:
            print("Could not save key file to: ", keyFile)
            exit(1)

    def readKeyFromFile(self, keyFile):
        """Reads key from a pickle file"""
        try:
            newPub = pickle.load(open(keyFile, "rb"))
            self.GPrime = newPub.GPrime
        except:
            print("Could not load key file from: ", keyFile)
            exit(1)

    def encrypt(self, m):
        """When given a message will encode"""
        # Error vector will be random
        z = random.randint(1, 7)
        c = bitFlip(modTwo(m * self.GPrime), z)
        return c

    def encryptFile(self, f):
        """Encrypts a whole file"""

        mf = open(f, "rb")
        m = mf.read(1)

        cf = open(f + ".ctxt", "wb")

        while m:
            # First half byte of message text
            m_1 = '{0:08b}'.format(ord(m))[0:4]
            m1_l = []
            c1 = ""
            for s in m_1:
                m1_l.append(s)
            m_1_m = np.matrix(m1_l, dtype=int)

            d1 = self.encrypt(m_1_m)
            for d in range(0, d1.size):
                c1 += str(d1.item(d))
            cf.write(chr(int(c1, 2)))

            # Second half byte of message text
            m_2 = '{0:08b}'.format(ord(m))[4:]
            m2_l = []
            c2 = ""
            for s in m_2:
                m2_l.append(s)
            m_2_m = np.matrix(m2_l, dtype=int)

            d2 = self.encrypt(m_2_m)
            for d in range(0, d2.size):
                c2 += str(d2.item(d))
            cf.write(chr(int(c2, 2)))

            m = mf.read(1)

        cf.close()
        mf.close()

    def dnaFileEncrypt(self, f, dlu):
        """Takes a files, encypts it and represents it as a DNA codeword"""
        cipherString = ""

        mf = open(f, "rb")
        m = mf.read(1)

        cf = open(f + ".dna.ctxt", "wb")

        while m:

            # First half byte of message text
            m_1 = '{0:08b}'.format(ord(m))[0:4]
            m1_l = []
            c1 = ""
            for s in m_1:
                m1_l.append(s)
            m_1_m = np.matrix(m1_l, dtype=int)

            d1 = self.encrypt(m_1_m)

            # Second half byte of message text
            m_2 = '{0:08b}'.format(ord(m))[4:]
            m2_l = []
            c2 = ""
            for s in m_2:
                m2_l.append(s)
            m_2_m = np.matrix(m2_l, dtype=int)
            d2 = self.encrypt(m_2_m)

            message = makeString(m_1_m) + makeString(m_2_m)
            cipherMessage = makeString(d1) + makeString(d2)
            cf.write(dlu.lookDNAEncrypt(cipherMessage[0:8]) + "\n")
            cf.write(dlu.lookDNAEncrypt(cipherMessage[8:]) + "\n")
            m = mf.read(1)

        cf.close()
        mf.close()


#### Brute Forcer ####
class bruteForcerH84():
    """Data structure that attempts to create Private Key from Given Public Key"""

    def __init__(self, GPrime):

        self.attempts = 0

        self.GPrime = GPrime

        self.GPrimeConsider = 0
        self.sConsider = 0
        self.pConsider = 0

        self.STries = list()
        self.PTries = list()

        self.G = np.matrix([
            [1, 0, 0, 0, 0, 1, 1, 1],
            [0, 1, 0, 0, 1, 0, 1, 1],
            [0, 0, 1, 0, 1, 1, 0, 1],
            [0, 0, 0, 1, 1, 1, 1, 0]
        ], dtype=int)

    def printCode(self):
        """Canonical print( self function"""
        print("Calculated GPrime:\n" + str(self.GPrimeConsider) + "\n")
        print("sM:\n" + str(self.sConsider) + "\n")
        print("pM:\n" + str(self.pConsider) + "\n")
        print("Attempts: " + str(self.attempts) + "\n")

    def attemptKey(self):
        """Attempts to reconstitute S,P given GPrime (G is already known)"""
        self.attempts = 1

        self.sConsider = genSMatrix(4)
        self.STries.append(self.sConsider)

        self.pConsider = genPMatrix(8)
        self.PTries.append(self.pConsider)

        self.GPrimeConsider = modTwo(self.sConsider * self.G * self.pConsider)

        while not np.array_equal(self.GPrimeConsider.A1, self.GPrime.A1):

            self.attempts += 1

            # Keep generating new S matrices to test
            self.sConsider = genSMatrix(4)
            while not checkOldGuesses(self.STries, self.sConsider):
                self.sConsider = genSMatrix(4)
            self.STries.append(self.sConsider)

            # Keep generating new P matrices to test
            self.pConsider = genPMatrix(8)
            while not checkOldGuesses(self.PTries, self.pConsider):
                self.pConsider = genPMatrix(8)
            self.PTries.append(self.pConsider)

            self.GPrimeConsider = modTwo(self.sConsider * self.G * self.pConsider)

        return True


#### Hamming 16,11 Version ######
#### Created to test security of Key Cracking ####

#### Private Key H1611 ####
class privateKeyH1611:
    """Datastructure to represent our Private Key"""

    def __init__(self, S=None, P=None):
        """Initalizer that will set S,P, random if not given values"""
        # Hamming 16,11 in standard
        self.G = np.matrix([
            [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
            [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
            [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
            [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1],
            [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1],
            [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1],
            [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0]
        ], dtype=int)
        self.H = np.matrix([
            [1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1],
            [0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1],
            [0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1],
            [0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0]
        ], dtype=int)

        # Can create these from known values, otherwise random
        if S == None:
            self.S = modTwo(genSMatrix(11))
        else:
            self.S = S

        if P == None:
            self.P = modTwo(genPMatrix(16))
        else:
            self.P = P

    def printCode(self):
        """Canonical print( to screen function"""
        print("S: \n" + str(self.S) + "\n")
        print("P: \n" + str(self.P) + "\n")
        print("GPrime: \n" + str(self.makeGPrime()) + "\n")

    def makeGPrime(self):
        """Returns a version of GPrime usable for calculations"""
        return modTwo(self.S * self.G * self.P)


#### Public Key H1611 ####
class publicKeyH1611:
    """Public Key Data Structure, much simplified then H8,4"""

    def __init__(self, GPrime):
        self.GPrime = GPrime

    def printCode(self):
        """Canonical print( to screen"""
        print("GPrime: \n" + str(self.GPrime) + "\n")


##### Brute Forcer Based on H16,11 ####
class bruteForcerH1611():
    """Data structure that attempts to create Private Key from Given Public Key"""

    def __init__(self, GPrime):

        self.attempts = 0

        self.GPrime = GPrime

        self.GPrimeConsider = 0
        self.sConsider = 0
        self.pConsider = 0

        self.STries = list()
        self.PTries = list()

        self.G = np.matrix([
            [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
            [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
            [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
            [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1],
            [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1],
            [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1],
            [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0]
        ], dtype=int)

    def printCode(self):
        """Canonical print( self function"""
        print("Calculated GPrime:\n" + str(self.GPrimeConsider) + "\n")
        print("sM:\n" + str(self.sConsider) + "\n")
        print("pM:\n" + str(self.pConsider) + "\n")
        print("Attempts: " + str(self.attempts) + "\n")

    def attemptKey(self):
        """Attempts to reconstitute S,P given GPrime (G is already known)"""
        self.attempts = 1

        self.sConsider = genSMatrix(11)
        self.STries.append(self.sConsider)

        self.pConsider = genPMatrix(16)
        self.PTries.append(self.pConsider)

        self.GPrimeConsider = modTwo(self.sConsider * self.G * self.pConsider)

        while not np.array_equal(self.GPrimeConsider.A1, self.GPrime.A1):
            self.attempts += 1

            self.sConsider = genSMatrix(11)
            while not checkOldGuesses(self.STries, self.sConsider):
                self.sConsider = genSMatrix(11)
            self.STries.append(self.sConsider)

            self.pConsider = genPMatrix(16)
            while not checkOldGuesses(self.PTries, self.pConsider):
                self.pConsider = genPMatrix(16)
            self.PTries.append(self.pConsider)

            self.GPrimeConsider = modTwo(self.sConsider * self.G * self.pConsider)

        return True


##### DNA Lookup Datastructure ###
class lookupDNA():
    """Data structure that keeps a table of DNA mappings"""

    def __init__(self, encryptFile, decryptFile):
        self.encLU = dict()
        self.decLU = dict()

        dreader = csv.reader(open(decryptFile, 'r'))
        for row in dreader:
            k, v = row
            self.decLU[k] = str(v)

        ereader = csv.reader(open(encryptFile, 'r'))
        for row in ereader:
            k, v = row
            self.encLU[k] = str(v)

    def lookDNAEncrypt(self, bstring):
        """Finds DNA sequence from 8 bits"""
        try:
            return str(self.encLU[bstring])
        except:
            return "?"

    def lookDNADecrypt(self, bstring):
        """Finds 8 bits from 8 characters of DNA"""
        try:
            return str(self.decLU[bstring])
        except:
            return "?"


if __name__ == "__main__":
    pass




## utils

In [8]:
import numpy as np

def string_to_block(msg, size):
  bits = ''.join(format(ord(i), '08b') for i in msg)
  blocks = []
  block = []
  for bit in bits:
    block.append(int(bit))
    if len(block) == size:
      blocks.append(np.matrix(block))
      block = []

  return blocks

def block_to_string_4(blocks):
    bits = ''.join(str(int(bit)) for block in blocks for bit in np.asarray(block).flatten())
    # Ensure we only process complete bytes
    bits = bits[:len(bits) - len(bits) % 8]
    chars = [chr(int(bits[i:i+8], 2)) for i in range(0, len(bits), 8)]
    return ''.join(chars)

def block_to_string_8(blocks):
    bits = ''.join(str(int(bit)) for block in blocks for bit in np.asarray(block).flatten())
    # Ensure we only process complete bytes
    bits = bits[:len(bits) - len(bits) % 8]
    chars = [chr(int(bits[i:i+8], 2)) for i in range(0, len(bits), 8)]
    return ''.join(chars)

In [9]:
print(block_to_string_8(string_to_block("H", 8)))

H


## example

### basic example

In [10]:
tPriv = privateKeyH84()
tPub = publicKeyH84(tPriv.makeGPrime())

m = np.matrix([0,1,1,0])
print(f"Original message {m}")

ct = tPub.encrypt(m)
print(f"Cypher message: {ct}")

mt = tPriv.decrypt(ct)
print(f"Decrypted message {mt}")

Original message [[0 1 1 0]]
Cypher message: [[0 0 0 1 1 0 0 1]]
Decrypted message [[0 1 1 0]]


### string example

In [11]:
tPriv = privateKeyH84()
tPub = publicKeyH84(tPriv.makeGPrime())

print("Generated Private Key:", tPriv)
print("Generated Public Key:", tPub)

# Message to encrypt
m = "Hello World!!!!"
print("\nOriginal message:", m)

# Convert message to 4-bit blocks
blocks = string_to_block(m,4)
print("4-bit blocks:", blocks)

# Encrypt the blocks
en_blocks = []
for b in blocks:
    encrypted = tPub.encrypt(b)
    en_blocks.append(encrypted)
    # print(f"Encrypting {b} -> {encrypted}")

# Convert encrypted 4-bit blocks into a string using 8-bit packing
print(en_blocks)
en_msg = block_to_string_8(en_blocks)
print("\nEncrypted message (as UTF-8 string):", en_msg)

# Convert encrypted message back to 8-bit binary blocks
en_blocks2 = string_to_block(en_msg,8)
print("Encrypted message as 8-bit binary blocks:", en_blocks2)

# Decrypt the blocks
de_blocks = []
for b in en_blocks2:
    decrypted = tPriv.decrypt(b)
    de_blocks.append(decrypted)
    # print(f"Decrypting {b} -> {decrypted}")

# Convert decrypted 4-bit blocks back to string
de_msg = block_to_string_4(de_blocks)
print("\nDecrypted message:", de_msg)

Generated Private Key: <__main__.privateKeyH84 object at 0x7d9f213b0210>
Generated Public Key: <__main__.publicKeyH84 object at 0x7d9f213a6d90>

Original message: Hello World!!!!
4-bit blocks: [matrix([[0, 1, 0, 0]]), matrix([[1, 0, 0, 0]]), matrix([[0, 1, 1, 0]]), matrix([[0, 1, 0, 1]]), matrix([[0, 1, 1, 0]]), matrix([[1, 1, 0, 0]]), matrix([[0, 1, 1, 0]]), matrix([[1, 1, 0, 0]]), matrix([[0, 1, 1, 0]]), matrix([[1, 1, 1, 1]]), matrix([[0, 0, 1, 0]]), matrix([[0, 0, 0, 0]]), matrix([[0, 1, 0, 1]]), matrix([[0, 1, 1, 1]]), matrix([[0, 1, 1, 0]]), matrix([[1, 1, 1, 1]]), matrix([[0, 1, 1, 1]]), matrix([[0, 0, 1, 0]]), matrix([[0, 1, 1, 0]]), matrix([[1, 1, 0, 0]]), matrix([[0, 1, 1, 0]]), matrix([[0, 1, 0, 0]]), matrix([[0, 0, 1, 0]]), matrix([[0, 0, 0, 1]]), matrix([[0, 0, 1, 0]]), matrix([[0, 0, 0, 1]]), matrix([[0, 0, 1, 0]]), matrix([[0, 0, 0, 1]]), matrix([[0, 0, 1, 0]]), matrix([[0, 0, 0, 1]])]
[matrix([[0, 0, 0, 1, 0, 1, 0, 1]]), matrix([[1, 0, 1, 1, 1, 1, 0, 0]]), matrix([[1, 0

# EC DSA signature

## algorithm

In [12]:
def find_inverse(number, modulus):
    return pow(number, -1, modulus)

class Point:
    def __init__(self, x, y, curve_config):
        a = curve_config['a']
        b = curve_config['b']
        p = curve_config['p']

        if (y ** 2) % p != (x ** 3 + a * x + b) % p:
            raise Exception("The point is not on the curve")

        self.x = x
        self.y = y
        self.curve_config = curve_config

    def is_equal_to(self, point):
        return self.x == point.x and self.y == point.y

    def add(self, point):
        p = self.curve_config['p']

        if self.is_equal_to(point):
            slope = (3 * point.x ** 2) * find_inverse(2 * point.y, p) % p
        else:
            slope = (point.y - self.y) * find_inverse(point.x - self.x, p) % p

        x = (slope ** 2 - point.x - self.x) % p
        y = (slope * (self.x - x) - self.y) % p
        return Point(x, y, self.curve_config)

    def multiply(self, times):
        current_point = self
        current_coefficient = 1

        pervious_points = []
        while current_coefficient < times:
            # store current point as a previous point
            pervious_points.append((current_coefficient, current_point))
            # if we can multiply our current point by 2, do it
            if 2 * current_coefficient <= times:
                current_point = current_point.add(current_point)
                current_coefficient = 2 * current_coefficient
            # if we can't multiply our current point by 2, let's find the biggest previous point to add to our point
            else:
                next_point = self
                next_coefficient = 1
                for (previous_coefficient, previous_point) in pervious_points:
                    if previous_coefficient + current_coefficient <= times:
                        if previous_point.x != current_point.x:
                            next_coefficient = previous_coefficient
                            next_point = previous_point
                current_point = current_point.add(next_point)
                current_coefficient = current_coefficient + next_coefficient

        return current_point

secp256k1_curve_config = {
    'a': 0,
    'b': 7,
    'p': 115792089237316195423570985008687907853269984665640564039457584007908834671663
}
x = 55066263022277343669578718895168534326250603453777594175500187360389116729240
y = 32670510020758816978083085130507043184471273380659243275938904335757337482424
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337
g_point = Point(x, y, secp256k1_curve_config)

def sign_message(message, private_key):
    k = random.randint(1, n)
    r_point = g_point.multiply(k)
    r = r_point.x % n
    if r == 0:
        return sign_message(message, private_key)
    k_inverse = find_inverse(k, n)
    s = k_inverse * (message + r * private_key) % n
    return r, s

def verify_signature(signature, message, public_key):
    (r, s) = signature
    s_inverse = find_inverse(s, n)
    u = message * s_inverse % n
    v = r * s_inverse % n
    c_point = g_point.multiply(u).add(public_key.multiply(v))
    return c_point.x == r

# test starts here
private_key = 123456789012345  # any random integer
public_key = g_point.multiply(private_key)
message = 12345  # any integer

signature = sign_message(message, private_key)
print('Signature: ', signature)
print('Is valid: ', verify_signature(signature, message, public_key))

Signature:  (112580045410092415765321967381999104636700501906542733597431023839966846365042, 114889848284798180017701582375910881024782061902203504273229113294794599249272)
Is valid:  True


## utils

In [13]:
def hash_message(message):
    return int(hashlib.sha256(message.encode()).hexdigest(), 16)

def sign_message_str(message, private_key):
    m = hash_message(message)
    return sign_message(m, private_key)

def verify_signature_str(signature, message, public_key):
    m = hash_message(message)
    return verify_signature(signature, m, public_key)

## example

### sign blowfish key

In [14]:
key,key_block = generate_max_blowfish_key()
key_int = int.from_bytes(key, byteorder='big')
key_bytes = key_int.to_bytes(56, byteorder='big')
print(f"Convert back check: {key==key_bytes}")

message = key_int
priv_key = random.randrange(start=1000)
pub_key = g_point.multiply(priv_key)
signature = sign_message(message, priv_key)
print('Signature: ', signature)
print('Is valid: ', verify_signature(signature, message, pub_key))

Convert back check: True
Signature:  (87239171160107078780552890804516690589330293108479496120136622865139789565689, 66626183655859858727268884066906385625146929963715963415855341896198397614479)
Is valid:  True


### sign string

In [15]:
priv_key = random.randrange(start=1000)
pub_key = g_point.multiply(priv_key)
msg = "Hello World!!"
print(msg)
signature = sign_message_str(msg, priv_key)
print('Signature: ', signature)
print('Is valid: ', verify_signature_str(signature, msg, pub_key))

Hello World!!
Signature:  (16447036446342696890295623789808638724851483813864392315955979456910669633364, 83400362275373959931386183044797659718443231621238492077071274808670026625166)
Is valid:  True


# interface

## utils

In [16]:
def blowfish_key_to_blocks(key):
    bit_list = []
    for byte in key:
        bits = [(byte >> i) & 1 for i in reversed(range(8))]
        bit_list.extend(bits)
    # Split into 4-bit blocks
    blocks = [bit_list[i:i+4] for i in range(0, len(bit_list), 4)]
    return blocks

def blocks_to_blowfish_key(blocks):
    bit_list = []

    for block in blocks:
        if isinstance(block, np.matrix):
            # Flatten matrix to a 1D list of integers
            values = np.asarray(block).flatten().tolist()
        else:
            values = block  # fallback if not a matrix

        # Append bits to bit_list
        bit_list.extend(values)

    # Group into 8-bit chunks
    byte_list = []
    for i in range(0, len(bit_list), 8):
        byte_bits = bit_list[i:i+8]
        if len(byte_bits) < 8:
            byte_bits += [0] * (8 - len(byte_bits))
        byte = sum(bit << (7 - idx) for idx, bit in enumerate(byte_bits))
        byte_list.append(byte)

    key = bytes(byte_list)
    blocks_32bit = [int.from_bytes(key[i:i+4], 'big') for i in range(0, len(key), 4)]

    return blocks_32bit


## classes

### Alice

In [17]:
class Alice:

  def __init__(self):
    self.blowfish_key,self.blowfish_key_blocks = generate_max_blowfish_key()
    self.sign_private_key = random.randrange(1000, 10**20)
    self.sign_public_key = g_point.multiply(self.sign_private_key)

    print()
    print(f"Alice - blowfish_key={self.blowfish_key}")
    print(f"Alice - sign_private_key={self.sign_private_key}")
    print(f"Alice - sign_public_key={self.sign_public_key}")

  def send_email(self, msg, bob):
    print(f"Alice - preparing message to send, message='{msg}'")

    # encrypt the message via blowfish
    bf = Blowfish(key_blocks=self.blowfish_key_blocks)
    msg_c = bf.encrypt_str(msg)

    # encrypt key with bob's key
    bob_public_key = bob.mcEliece_public_key
    k_blocks = blowfish_key_to_blocks(self.blowfish_key)
    msg_ck = []
    for b in k_blocks:
        ck_b = bob_public_key.encrypt(b)
        msg_ck.append(ck_b)

    msg_s = sign_message_str(msg_c, self.sign_private_key)

    print("Alice - prepared email for Bob")
    print(f"\tmsg_c='{msg_c}'")
    print(f"\tmsg_ck='{msg_ck}'")
    print(f"\tmsg_s='{msg_s}'")

    print("Alice - Sending email to Bob")
    reply = bob.receive_email(msg_c, msg_ck, msg_s, self)
    print()
    print(f"Alice - received reply from Bob: {reply}")


### Bob

In [18]:
class Bob:

  def __init__(self):
    self.mcEliece_private_key = privateKeyH84()
    self.mcEliece_public_key = publicKeyH84(self.mcEliece_private_key.makeGPrime())
    print(f"Bob - mcEliece_private_key={self.mcEliece_private_key}")
    print(f"Bob - mcEliece_public_key={self.mcEliece_public_key}")

  def receive_email(self, msg_c, msg_ck, msg_s, sender):
    print()
    print("Bob - received email")
    print(f"\tmsg_c='{msg_c}'")
    print(f"\tmsg_ck='{msg_ck}'")
    print(f"\tmsg_s='{msg_s}'")

    # verify signsinure
    sender_sign_public_key = sender.sign_public_key
    verify_res = verify_signature_str(msg_s, msg_c, sender_sign_public_key)
    if (verify_res):
      print("Bob - message verified successfully")
    else:
      print("Bob - verification failed")
      return "VERIFICATION FAILED"

    # decrypt key
    msg_k = []
    for b in msg_ck:
        k_b = self.mcEliece_private_key.decrypt(b)
        msg_k.append(k_b)
    bf_k_blocks = blocks_to_blowfish_key(msg_k)

    # decrypt message
    bf = Blowfish(key_blocks=bf_k_blocks)
    msg_e = bf.decrypt_str(msg_c)
    print(f"Bob - decrypted message: '{msg_e}'")
    return "SUCCESS"

## main

In [19]:
alice = Alice()
bob = Bob()

print()
msg = input("Enter message to send: ")
alice.send_email(msg, bob)


Alice - blowfish_key=b'5\t\xea!\x10\xec\x00B\xc8k\xe8<e8_\xf9\xc3`\xc5\x84e=\x8d\x00\x9d\x84\xcd\xf9\xdf\xff\xbf\x07\xa2\xcbXe\r\xb5$<\xfe]\x82\xfd\xed\xb3\xa8\x94"BHp\x1d\x96\xf7\xb3'
Alice - sign_private_key=45090183623643629041
Alice - sign_public_key=<__main__.Point object at 0x7d9f21380d10>
Bob - mcEliece_private_key=<__main__.privateKeyH84 object at 0x7d9f212ab690>
Bob - mcEliece_public_key=<__main__.publicKeyH84 object at 0x7d9f447da8d0>

Enter message to send: Hello there!!!
Alice - preparing message to send, message='Hello there!!!'
Alice - prepared email for Bob
	msg_c='૪հೆ⅚᱊━௲ஜ⇂᪂፠२Ⅸϟ'
	msg_ck='[matrix([[0, 1, 1, 0, 1, 1, 1, 0]]), matrix([[0, 0, 0, 0, 1, 1, 0, 1]]), matrix([[0, 1, 0, 0, 0, 0, 0, 0]]), matrix([[1, 1, 0, 1, 1, 0, 1, 0]]), matrix([[1, 0, 0, 1, 1, 1, 1, 0]]), matrix([[1, 0, 1, 1, 0, 0, 0, 0]]), matrix([[0, 1, 1, 0, 1, 0, 1, 1]]), matrix([[0, 0, 1, 0, 1, 1, 1, 1]]), matrix([[0, 0, 1, 1, 0, 1, 1, 1]]), matrix([[0, 0, 0, 0, 0, 1, 0, 0]]), matrix([[0, 0, 0, 1, 1, 1